# Redes Neuronales - Trabajo Práctico N° 1 - Notebook #2
En esta segunda notebook, se busca definir cuál métrica es más apropiada para analizar la performance del modelo y qué hiper parámetros se van a utilizar para el ajuste del modelo acorde a la validación. Finalmente, estas decisiones se vuelcan en la selección del mejor modelo para el problema de la clasificación de correos electrónicos asociados grupos de noticias.

### Integrantes del grupo
* Kammann, Lucas Agustín
* Gaytan, Joaquín Oscar

# 1. Métrica
La métrica a utilizar para cuantificar la performance de los modelos, seleccionar los hiperparámetros y validarlos será la **exactitud** o **accuracy**.

## 1.1 Justificación
La problemática a resolver tiene por objetivo asegurar clasificar entre múltiples clases, por ende, el objetivo es acertar la mayor cantidad de predicciones posibles. Para este tipo de problemas, conviene usar la exactitud, pero antes es necesario comprobar que la distribución de clases está balanceada o es uniforme, dado que si no fuera así (o aproximadamente así) entonces habría un sesgo en la estimación. Esto último se debe a que no seríamos capaces de cuantificar realmente lo malo que el modelo es prediciendo aquellas clases minoritarias.

Es decir, si bien es una métrica acorde al problema, cuando las clases no están balanceadas su interpretación numérica no es realista. En esos casos, se puede utilizar el promedio de la sensibilidad de cada clase, porque dicha sensibilidad representa la probabilidad de acertar en la predicción dada cada clase y si luego las promediamos estamos ponderando de igual forma cada clase.

En conclusión, dado lo que se observó en previos análisis **(ver Notebook #1)**, se puede asumir que la distribución de clases es aproximadamente uniforme por lo cual la exactitud es una métrica aceptable. Si se quisiera obtener una cantidad más realista, con el promedio de sensibilidad se cumpliría tal objetivo.

# 2. Hiper parámetros
Se consideran hiper parámetros aquellos que se determinan de manera óptima eligiendo aquel que da mejor resultado en el conjunto de datos de validación, donde muchos modelos con diversos tipos y valores de hiper parámetros compiten por ver cuál obtuvo la mejor medida de performance, es decir, de la métrica. Para ello, consideraremos como hiper parámetros los siguientes aspectos,

* Algoritmo de preprocesamiento empleado: Stemming, Lemmatization, Ninguno
* Filtrado por stop words
* Vectorizer empleado: CountVectorizer, TfIdfVectorizer
* Modelo empleado: MultinomialNaiveBayes, OneVsRestClassifier
* Coeficiente de Laplacian Smoothing
* Coeficiente de mínima frecuencia por documentos
* Coeficiente de máxima frecuencia por documentos

*NOTA*, sólo correr las siguientes celdas de preprocesamiento, si no se poseen los '*.txt' preprocesados.

## 2.1. Algoritmos de preprocesamiento
A continuación, se corren los algoritmos de procesamiento directamente sobre el conjunto de entreamiento para evitar tener que realizar el preprocesamiento cada vez teniendo en cuenta que no existen más opciones a analizar.

### 2.1.1. Descargando el dataset

In [7]:
import pickle

In [1]:
from sklearn.datasets import fetch_20newsgroups
import numpy as np

# Loading the train dataset
train = fetch_20newsgroups(
    subset='train', 
    shuffle=True, 
    remove=('headers', 'footers')
)

# Train dataset, casting to numpy array
train_raw_input = np.array(train.data)
train_output = np.array(train.target)
train_size = len(train_raw_input)

# Logging useful information
print(f'Dataset Train: {train_size} elements')

Dataset Train: 11314 elements


### 2.1.2. Dataset original

In [24]:
%%time

from nltk import word_tokenize

# Process and save the stemmed trainning data
normal_train_raw_input = []
for document in train_raw_input:
    tokens = word_tokenize(document)
    new_document = " ".join([token.lower() for token in tokens if token.isalpha()])
    normal_train_raw_input.append(new_document)

# Logging
print('Normal pre processing algorithm finished.')

Normal pre processing algorithm finished.
Wall time: 42 s


In [25]:
%%time

# Create the structure of the trainning data set for the non-processed case
normal_train = {
    'input': train_raw_input,
    'output': train_output
}

# Save with pickle
with open('tp1_ej1_train_normal.txt', 'wb') as file:
    pickle.dump(normal_train, file)

# Logging
print('The normal trainning dataset has been saved in the local storage system.')

The normal trainning dataset has been saved in the local storage system.
Wall time: 48.3 s


### 2.1.3. Dataset con stemming

In [26]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to C:\Users\Lucas A.
[nltk_data]     Kammann\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Lucas A.
[nltk_data]     Kammann\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [27]:
%%time

from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize

# Instantiate the stemmer instance
stemmer = PorterStemmer()

# Process and save the stemmed trainning data
stemmed_train_raw_input = []
for document in train_raw_input:
    tokens = word_tokenize(document)
    new_document = " ".join([stemmer.stem(token.lower()) for token in tokens if token.isalpha()])
    stemmed_train_raw_input.append(new_document)

# Logging
print('Stemming pre processing algorithm finished.')

Stemming pre processing algorithm finished.
Wall time: 1min 27s


In [28]:
%%time

# Create the structure of the trainning data set for the stemming case
stemmed_train = {
    'input': stemmed_train_raw_input,
    'output': train_output
}

# Save with pickle
with open('tp1_ej1_train_stemmed.txt', 'wb') as file:
    pickle.dump(stemmed_train, file)

# Logging
print('The stemming trainning dataset has been saved in the local storage system.')

The stemming trainning dataset has been saved in the local storage system.
Wall time: 154 ms


### 2.1.4. Dataset con lemmatization

In [29]:
%%time

from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

# Instantiate the lemmatizer instance
lemmatizer = WordNetLemmatizer()

# Process and save the lemmatized trainning data
lemmatized_train_raw_input = []
for document in train_raw_input:
    tokens = word_tokenize(document)
    new_document = " ".join([lemmatizer.lemmatize(token.lower()) for token in tokens if token.isalpha()])
    lemmatized_train_raw_input.append(new_document)

# Logging
print('Lemmatization pre processing algorithm finished.')

Lemmatization pre processing algorithm finished.
Wall time: 42.8 s


In [30]:
%%time

# Create the structure of the trainning data set for the lemmatization case
lemmatization_train = {
    'input': lemmatized_train_raw_input,
    'output': train_output
}

# Save with pickle
with open('tp1_ej1_train_lemmatization.txt', 'wb') as file:
    pickle.dump(lemmatization_train, file)

# Logging
print('The lemmatization trainning dataset has been saved in the local storage system.')

The lemmatization trainning dataset has been saved in the local storage system.
Wall time: 72.5 ms


# Preprocesamiento de los datos

In [12]:
from sklearn.datasets import fetch_20newsgroups
from sklearn import metrics
import numpy as np

# Loading the datasets
train = fetch_20newsgroups(
    subset='train', 
    shuffle=True, 
    remove=('headers', 'footers', 'quotes')
)

test = fetch_20newsgroups(
    subset='test', 
    shuffle=True, 
    remove=('headers', 'footers', 'quotes')
)

# Train dataset, casting to numpy array
train_raw_input = np.array(train.data)
train_output = np.array(train.target)
train_size = len(train_raw_input)

# Test dataset, casting to numpy array
test_raw_input = np.array(test.data)
test_output = np.array(test.target)
test_size = len(test_raw_input)

# Logging useful information
print(f'Dataset Train: {train_size} elements')
print(f'Dataset Test: {test_size} elements')

Dataset Train: 11314 elements
Dataset Test: 7532 elements


In [34]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Preprocessing
vectorizer = CountVectorizer(encoding="latin1")
train_input = vectorizer.fit_transform(train_raw_input)
test_input = vectorizer.transform(test_raw_input)

# Entrenamiento

In [40]:
from src.multinomial_naive_bayes import MultinomialNaiveBayes

# Training the multinomial naive bayes model
classifier = MultinomialNaiveBayes(alpha=0.01)
classifier.fit(train_input, train_output)

# Validación del modelo

In [41]:
predictions = classifier.predict(test_input.todense())

In [42]:
accuracy = metrics.accuracy_score(test_output, predictions)

In [43]:
recall = metrics.recall_score(test_output, predictions, average='macro')

In [44]:
print(f'Accuracy: {accuracy}')
print(f'Balanced accuracy: {recall}')

Accuracy: 0.6460435475305364
Balanced accuracy: 0.6352908156554821
